## Web Scraping using Selenium

Importing the required libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time

In [2]:
def search_url(url, page_number):
    template = 'https://www.myntra.com/lipstick?rawQuery={}&p={}'
    return template.format(url, page_number)

driver = webdriver.Chrome()

org_url = input('enter your search term: ')

brands = []
price=[]
original_price=[]
description=[]
ratings=[]
product_url=[]

for i in range(1, 11):
    driver.get(search_url(org_url, i))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        brand = soup.find_all('h3', class_="product-brand")
        for a in brand:
            brands.append(a.text)
    except AttributeError:
        continue
        
    #price
    try:
        pr = soup.find_all('span',class_="product-discountedPrice")
        for b in pr:
            price.append(int(b.text.strip('Rs. ')))
    except AttributeError:
        price.append(None)
        
    missing_count = len(brands) - len(price)
    if missing_count > 0:
        price.extend([None]*missing_count)
    
    #original price
    try:
        mrp = soup.find_all('span', class_ = 'product-strike')
        for c in mrp:
            original_price.append(int(c.text.strip('Rs. ')))
    except AttributeError:
        original_price.append(None)
        
    missing_counts = len(brands) - len(original_price)
    if missing_counts > 0:
        original_price.extend([None]*missing_counts)
    
    #description
    try:
        des = soup.find_all('h4', class_='product-product')
        description.extend([i.text for i in des])
    except AttributeError:
        description =' '
        
    #product url

    try:
        li_elements = soup.find_all('li', class_="product-base")
        for d in li_elements:
            a_elements = d.find_all('a', {'data-refreshpage': 'true', 'target': '_blank'})
            for a in a_elements:
                href = 'http://myntra.com/' + a['href']
                product_url.append(href)
    except AttributeError:
        product_url=' '

        
driver.close()

enter your search term: lipstick


In [3]:
df = pd.DataFrame(columns=['brand_name','price','original_price','description','product_url'])
df['brand_name'] = brands
df['price']=price
df['original_price']=original_price
df['description']=description
df['product_url']=product_url

In [4]:
df

,brand_name,price,original_price,description,product_url
0,ELLE 18,93.0,100.0,Lip Color -Berry Tea W14,http://myntra.com/lipstick/elle+18/elle-18--co...
1,ELLE 18,93.0,100.0,Matte Lip Color Crazy Red R31,http://myntra.com/lipstick/elle+18/elle-18-col...
2,SUGAR POP,239.0,299.0,Satin Matte Lipstick- Taupe 01,http://myntra.com/lipstick/sugar+pop/sugar-pop...
3,ELLE 18,93.0,100.0,Lip Color Maroon Silk R37,http://myntra.com/lipstick/elle+18/elle-18-col...
4,Blue Heaven,245.0,350.0,Set of 3 Candy Crush Lip Color,http://myntra.com/lipstick/blue+heaven/blue-he...
...,...,...,...,...,...
495,ELLE 18,NaN,NaN,Liquid LipColor - Warm Fusicia,http://myntra.com/lipstick/elle+18/elle-18-one...
496,MARS,NaN,NaN,Double Trouble Lip Crayon - 05,http://myntra.com/lipstick/mars/mars-double-tr...
497,Colors Queen,NaN,NaN,Luxury Matte Lipstick - 03,http://myntra.com/lipstick/colors+queen/colors...
498,M.A.C,NaN,NaN,Matte Lipstick - Soar,http://myntra.com/lipstick/m.a.c/mac-matte-lip...


In [5]:
len(brands)

500

In [6]:
df.to_csv('myntra_products_lipsticks.csv')

In [8]:
df['brand_name'].nunique()

61

In [10]:
df['brand_name'].unique()

array(['ELLE 18', 'SUGAR POP', 'Blue Heaven', 'Lakme',
       'NYX PROFESSIONAL MAKEUP', 'krayons', 'Clinique', 'Renee',
       'Lovechild Masaba', 'Olivia', 'PERPAA', 'BIOTIQUE NATURAL MAKEUP',
       'Iba', 'SERY', 'Deborah', 'Plum', 'PUPA MILANO', 'Stars Cosmetics',
       'brwn. pro melanin care', 'FORFOR', 'Maybelline', 'LOreal',
       'CAL Losangeles', 'ForSure', 'Marie Claire', 'FACES CANADA',
       'SWISS BEAUTY', 'SUGAR', 'MARS', 'Earth Rhythm', 'Colorbar',
       'M.A.C', 'Mamaearth', 'Revlon', 'peripera', 'CVB', 'INCOLOR',
       'The Face Shop', 'ETUDE', 'Iba HALAL CARE', 'LOVE EARTH',
       'MyGlamm', 'Colors Queen', 'Just Herbs', 'Chambor', 'CurlyTales',
       'THE BODY SHOP', 'Makeup Revolution London', 'Recode', 'CARA MIA',
       'KIRO', 'Seven Seas', 'July', 'Praush', 'Ronzille',
       'Rimmel London', 'KLENVOR', 'Milagro beauty', 'Adbeni',
       'Bobbi Brown', 'Fashion Colour'], dtype=object)

###### Similarly, other products details can be extracted for further analysis